# Clasificator pentru amprentele digitale

Sistem de clasificare a amprentelor digitale în trei categorii

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

In [ ]:
!ls /content/drive/MyDrive/Facultate\ Informatica/Profesor/2024\ -\ 2025/Sisteme expert\ si\ metode\ biometrice\ in\ securitatea\ informatiei/Curs/ColabMount/DATA/

NISTDB4_RAW  test_deformed_images


In [ ]:
datadir = '/content/drive/MyDrive/Facultate Informatica/Profesor/2024 - 2025/Sisteme expert si metode biometrice in securitatea informatiei/Curs/ColabMount/DATA/NISTDB4_RAW/train_set'
files = os.listdir(datadir)
print(files)

['class1_arc', 'class2_whorl', 'class3_loop']


In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

## Reteaua neuronala

Această rețea este simplă, dar eficientă pentru sarcini de clasificare a imaginilor de amprente în mai multe categorii (de exemplu, clasificare biometrică). Rețeaua poate fi extinsă sau modificată cu ușurință pentru a se adapta altor dimensiuni de intrare, alte dimensiuni ale embedding-ului sau un număr diferit de clase.

Rețeaua procesează imagini în tonuri de gri (un singur canal) de dimensiune pătrată (implicit 128x128 pixeli) și clasifică fiecare imagine într-una dintre cele num_classes (implicit 3 clase posibile).

Blocul convoluțional conține trei straturi convoluționale, fiecare urmat de o funcție de activare ReLU. După al doilea și al treilea strat convoluțional se aplică pooling (MaxPool2d) pentru reducerea dimensiunii. Dimensiunile canalelor cresc astfel: 1 → 32 → 64 → 128, adaptând progresiv reprezentarea imaginii.

Ultimul strat are 32 de pixeli, un stat relativ mare, datorită structurilor relativ reduse ale amprentelor.

După straturile convoluționale, ieșirea este aplatizată (Flatten). Aceasta este urmată de un strat Linear care generează un spațiu de embedding (implicit 512 dimensiuni), urmat de ReLU. Ultimul strat este un strat linear care generează logits pentru fiecare clasă (nu se aplică sigmoid deoarece se presupune o clasificare multi-clasă).

In [ ]:
class FingerprintClassifier(nn.Module):
    def __init__(self, image_size=128, embedding_size=512, num_classes=3):
        super(FingerprintClassifier, self).__init__()
        self.image_size = image_size

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),  # 1 input channel for grayscale
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_output_size = self._get_conv_output_size()

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.conv_output_size, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, num_classes)  # No sigmoid for multi-class logits
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

    def _get_conv_output_size(self):
        dummy_input = torch.ones(1, 1, self.image_size, self.image_size)  # 1 channel for grayscale
        x = self.conv_layers(dummy_input)
        return x.numel()


## Dataset

Dataset-ul citeste subdirectoarele setului de date de antrenament. Fiecare subdirector reprezintă o anumită clasă:

- class1_arc
- class2_whorl
- class3_loop

In timpul inițializării setului de date, fiecare imagine este marcată cu un label (exprimat ca număr întreg pe baza dictionarului label_map).

Funcția __getitem__ returnează perechea imagine (în format PIL) si clasa din care face parte (sub forma unui numar intreg).

In [ ]:
class FingerprintClassificationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_map = {
            'class1_arc': 0,
            'class2_whorl': 1,
            'class3_loop': 2
        }

        for class_name, label in self.label_map.items():
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                for file in os.listdir(class_dir):
                    if file.endswith(('.png', '.jpg', '.jpeg')):
                        self.image_paths.append(os.path.join(class_dir, file))
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

## Funcția de cost

Funcția de cost este aceeași pe care o folosim la clasificări: CrossEntropyLoss

In [ ]:
class ClassificationLoss(nn.Module):
    def __init__(self):
        super(ClassificationLoss, self).__init__()
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        return self.loss_fn(outputs, targets)

## Codul de antrenament

In [ ]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    dataset = FingerprintClassificationDataset(
        root_dir=datadir,
        transform=transform
    )

    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    model = FingerprintClassifier().to(device)
    criterion = ClassificationLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        loop = tqdm(dataloader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=False)
        for images, labels in loop:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(dataloader)}")

    torch.save(model.state_dict(), "fingerprint_classifier.pth")
    print("Training complete. Model saved.")

In [ ]:
train()

Epoch 1, Loss: 1.2023366769154866


Epoch 2, Loss: 1.020963538260687


Epoch 3, Loss: 0.7276779430253165


Epoch 4, Loss: 0.5604939377024061


Epoch 5, Loss: 0.4688888952845619


Epoch 6, Loss: 0.4189388382292929


Epoch 7, Loss: 0.3513593460122744


Epoch 8, Loss: 0.22625251458514303


Epoch 9, Loss: 0.1756809168450889


Epoch 10, Loss: 0.08808367283615683
Training complete. Model saved.
